In [ ]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
from IPython.display import clear_output
%matplotlib inline
from warnings import simplefilter
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
rooth_path="UrbanSound8K/image/"

In [ ]:
extracted_features =[]
for root, directories, files in os.walk(root_path):
    for file_name in files:
        
        if root is not root_path and file_name.endswith('.png'):
           data = get_spectrogram(os.path.join(root,file_name))
           isplitted = file_name.split("-")
           extracted_features.append([data,isplitted[1]])